# TFIDF (Word, BPE, Weighted Embeddings)

## Before You Start

### Libraries

In [ ]:
# install the essential libraries that are not available on google colab
!pip install youtokentome
! pip install gdown
# fasttext
!git clone https://github.com/facebookresearch/fastText.git 
!cd fastText && pip install .

In [3]:
# import libraries 
import os
import numpy as np
import codecs
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import youtokentome as yttm
import fasttext.util
import fasttext
import yaml

In [4]:
# create the utils folder and upload evaluation.ipynb and preprocess.ipynb
# ! mkdir utils
%run "./utils/evaluation.ipynb"
%run "./utils/preprocess.ipynb"

### Get The Skip-gram Fasttext Model

In [5]:
# https://github.com/taesiri/PersianWordVectors
SKIPGRAM_MODEL_FILE_ID = '1wPnMG9_GNUVdSgbznQziQc5nMWI3QKNz'

In [6]:
# Let's explore the SKIPGRAM Model

!gdown --id $SKIPGRAM_MODEL_FILE_ID 

Downloading...
From: https://drive.google.com/uc?id=1wPnMG9_GNUVdSgbznQziQc5nMWI3QKNz
To: /content/farsi-dedup-skipgram.bin
100% 4.37G/4.37G [01:13<00:00, 38.2MB/s]


In [7]:
# loading the Model
model_skipgram = fasttext.load_model('farsi-dedup-skipgram.bin')

# official persian fasttext
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz
# !gunzip cc.fa.300.bin.gz
# model_skipgram = fasttext.load_model('cc.fa.300.bin')

### Download Data

In [ ]:
! wget https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/evaluation_IR.yml -P ./data
! wget https://github.com/language-ml/2-LM-embedding-projects/raw/main/problem3/doc_collection.zip -P ./data
! unzip ./data/doc_collection.zip -d ./data
! wget https://raw.githubusercontent.com/SajjadMb/Masnavi_NLP_Toolkit/main/stopwords.txt -P ./data

## Preprocess

### Read data

set path of data in `PATH` variable and ground truth in `YAML_PATH` variable

In [9]:
PATH = './data/IR_dataset/'
YAML_PATH = './data/evaluation_IR.yml/'

PATH = PATH.rstrip('/')
YAML_PATH = YAML_PATH.rstrip('/')

store txt files into a list named `doc`

In [10]:
docs = []
for index in tqdm(range(0, 3258)):
    with open(f"{PATH}/{index}.txt", 'r', encoding='utf8') as file_reader:
        docs.append(file_reader.read())

100%|██████████| 3258/3258 [00:00<00:00, 18626.94it/s]


show example of data

In [11]:
for doc in docs[:3]:
    print (doc[:500])
    print('-' * 50)

برخی از هواداران مصدق یا اعضای جبهه ملی که در زمان نخست‌وزیری مصدق، از جبهه ملی یا از هیئت‌وزیران کنار گذاشته شده یا کنار رفتند، پس از جدایی از مصدق، به انتقاد از کارنامه وی پرداختند و حتی برای سرنگونی‌اش تلاش کردند. برخی از این افراد عبارت‌اند از:
* فضل‌الله زاهدی (نخست‌وزیر کودتا)
* علی امینی
* حسین مکی، (که در آغاز سرباز فداکار وطن نامیده شد ولی در پایان به دلیل مخالفت با مصدق از سوی هواداران جبهه ملی سرباز خطاکار وطن خطاب می‌شد)
* مظفر بقایی (به دلیل اتهام مشارکت در قتل سرتیپ افشار طوس و سپس
--------------------------------------------------
جبهه ملی ایران که به اختصار جبهه ملی نیز خوانده می‌شود سازمان سیاسی ملی‌گرای سکولار، دموکرات و جمهوری‌خواه فعال در ایران است، که در حال حاضر تحت رهبری دکتر موسویان فعالیت می‌کند.

جبهه ملی ایران در سال ۱۳۲۸ توسط سیاستمدارانی از قبیل محمد مصدق، حسین فاطمی و کریم سنجابی تأسیس شد و به پیش نهاد حسین فاطمی، ملی شدن صنعت نفت ایران را مطرح کرد. دکتر مصدق در کتاب خاطرات خود می‌گوید: «پیشنهاد ملی‌شدن صنعت نفت در سراسر کشور ابتکار شادروان دکتر حسین فاطمی

### Data clean

In [12]:
# cleaned data
docs = [clean_text(doc, sentence=False, only_persian=True) for doc in tqdm(docs)]
for doc in docs[:3]:
    print (doc[:500])
    print('-' * 50)

100%|██████████| 3258/3258 [00:03<00:00, 1085.68it/s]

برخی از هواداران مصدق یا اعضای جبهه ملی که در زمان نخست وزیری مصدق از جبهه ملی یا از هییت وزیران کنار گذاشته شده یا کنار رفتند پس از جدایی از مصدق به انتقاد از کارنامه وی پرداختند و حتی برای سرنگونی اش تلاش کردند برخی از این افراد عبارت اند از فضل الله زاهدی نخست وزیر کودتا علی امینی حسین مکی که در آغاز سرباز فداکار وطن نامیده شد ولی در پایان به دلیل مخالفت با مصدق از سوی هواداران جبهه ملی سرباز خطاکار وطن خطاب می شد مظفر بقایی به دلیل اتهام مشارکت در قتل سرتیپ افشار طوس و سپس اتهام شرکت در کودت
--------------------------------------------------
جبهه ملی ایران که به اختصار جبهه ملی نیز خوانده می شود سازمان سیاسی ملی گرای سکولار دموکرات و جمهوری خواه فعال در ایران است که در حال حاضر تحت رهبری دکتر موسویان فعالیت می کند جبهه ملی ایران در سال توسط سیاستمدارانی از قبیل محمد مصدق حسین فاطمی و کریم سنجابی تاسیس شد و به پیش نهاد حسین فاطمی ملی شدن صنعت نفت ایران را مطرح کرد دکتر مصدق در کتاب خاطرات خود می گوید پیشنهاد ملی شدن صنعت نفت در سراسر کشور ابتکار شادروان دکتر حسین فاطمی بود با پیشنها

### BPE

In [13]:
# Set path of bpe model and corpus that will be created by train cell
BPE_PATH = './bpe/'
BPE_PATH = BPE_PATH.rstrip('/')
os.makedirs(BPE_PATH, exist_ok=True)
train_data_path = BPE_PATH + "/train_data.txt"
model_path = BPE_PATH + "/bpe.model"

#### Train

In [14]:
# Generating corpus file with training data
pretrained_bpe_texts = ' '.join(docs)

with open(train_data_path, "w") as w_file:
    w_file.write(pretrained_bpe_texts)

# Training model
yttm.BPE.train(data=train_data_path, vocab_size=5000, model=model_path, unk_id=1, bos_id=2, eos_id=3)

#### Load and Test

In [15]:
# Loading model
bpe = yttm.BPE(model=model_path)

# first document as a test 
test_text = docs[0]

# Tokenization test
print(bpe.encode(test_text, output_type=yttm.OutputType.SUBWORD))

['▁برخی', '▁از', '▁هواداران', '▁مصدق', '▁یا', '▁اعضای', '▁جبهه', '▁ملی', '▁که', '▁در', '▁زمان', '▁نخست', '▁وزیری', '▁مصدق', '▁از', '▁جبهه', '▁ملی', '▁یا', '▁از', '▁هییت', '▁وزیر', 'ان', '▁کنار', '▁گذاشته', '▁شده', '▁یا', '▁کنار', '▁رفتند', '▁پس', '▁از', '▁جدایی', '▁از', '▁مصدق', '▁به', '▁انتقاد', '▁از', '▁کار', 'نامه', '▁وی', '▁پرداختند', '▁و', '▁حتی', '▁برای', '▁سرنگ', 'ونی', '▁اش', '▁تلاش', '▁کردند', '▁برخی', '▁از', '▁این', '▁افراد', '▁عبارت', '▁اند', '▁از', '▁فضل', '▁الله', '▁ز', 'اه', 'دی', '▁نخست', '▁وزیر', '▁کودتا', '▁علی', '▁ام', 'ینی', '▁حسین', '▁م', 'کی', '▁که', '▁در', '▁آغاز', '▁سرباز', '▁ف', 'دا', 'کار', '▁و', 'طن', '▁نامیده', '▁شد', '▁ولی', '▁در', '▁پایان', '▁به', '▁دلیل', '▁مخالفت', '▁با', '▁مصدق', '▁از', '▁سوی', '▁هواداران', '▁جبهه', '▁ملی', '▁سرباز', '▁خط', 'اک', 'ار', '▁و', 'طن', '▁خطاب', '▁می', '▁شد', '▁مظ', 'فر', '▁ب', 'قایی', '▁به', '▁دلیل', '▁اتهام', '▁مشارکت', '▁در', '▁قتل', '▁سر', 'تی', 'پ', '▁افشار', '▁ط', 'وس', '▁و', '▁سپس', '▁اتهام', '▁شرکت', '▁در', '▁کودتای', 

## TFIDF TOP K Relevant Documents 

In [16]:
# TFIDF Class: a class to generate TFIDF for your docs and get top k relvant documents
class tfidf:
    
    def __init__(self, docs, ngram_range, bpe_model=None):
        self._min_df = 1
        self._max_df=0.5
        self._max_features=10000
        self._docs = docs
        self._bpe = bpe_model
        self._ngram_range = ngram_range
        if self._bpe:
            print("we are using bpe")
            self._model_tfidf = TfidfVectorizer(analyzer="word", min_df=self._min_df, max_df=self._max_df, max_features=self._max_features, ngram_range=self._ngram_range, tokenizer= lambda x: self.bpe_tokenizer(x))
        else:
            self._model_tfidf = TfidfVectorizer(analyzer="word", min_df=self._min_df, max_df=self._max_df, max_features=self._max_features, ngram_range=self._ngram_range)

        self._matrix = self._model_tfidf.fit_transform(docs)
        self._feature_names = self._model_tfidf.get_feature_names_out()
        #Load skip-gram embedding vector for each TF-IDF term
        self._tfidf_emb_vecs = np.vstack([model_skipgram.get_word_vector(word) for word in self._feature_names])

        #To get a TF-IDF weighted skip-gram vector summary of each document, 
        #we just need to matrix multiply docs_vecs with tfidf_emb_vecs
        self._docs_emb = np.dot(self._matrix.toarray(), self._tfidf_emb_vecs)


    def bpe_tokenizer(self, text):
        tokens = self._bpe.encode(text, output_type=yttm.OutputType.SUBWORD)
        return tokens 
        

    def tfidf_top_k(self, query, k=2):
        
        query_tfidf = self._model_tfidf.transform([query])

        # Stores cosine similarity scores
        doc_scores = []

        # Compute the cosine similarity scores
        for doc in self._matrix:
            doc_scores.append(cosine_similarity(query_tfidf, doc)[0][0])

        # Sort list of doc_scores and return the top k indices of highest scores
        sorted_scores = sorted(enumerate(doc_scores), key=lambda ind_score: ind_score[1], reverse=True)
        if k!=-1:
            top_doc_indices = [ind for ind, score in sorted_scores[:k]]
        else:
            top_doc_indices=sorted_scores
        return top_doc_indices



    def tfidf_weighted_top_k(self, query, k=2):

        query_tfidf = self._model_tfidf.transform([query])
        query_emb = np.dot(query_tfidf.toarray(), self._tfidf_emb_vecs)
        # Stores cosine similarity scores
        doc_scores = []

        # Compute the cosine similarity scores
        for doc in self._docs_emb:
            doc_scores.append(cosine_similarity(query_emb, doc.reshape(1, -1))[0][0])

        # Sort list of doc_scores and return the top k indices of highest scores
        sorted_scores = sorted(enumerate(doc_scores), key=lambda ind_score: ind_score[1], reverse=True)
        if k!=-1:
            top_doc_indices = [ind for ind, score in sorted_scores[:k]]
        else:
            top_doc_indices = sorted_scores
        return top_doc_indices

In [17]:
query = "آدولف هیتلر شکست و مرگ"

###  Word Level: ngram (1,1)

In [18]:
tf_obj_word_1_1 = tfidf(docs=docs, ngram_range=(1,1), bpe_model=None)
# show 1000 features among 10000 selected features by tfidf 
print(tf_obj_word_1_1._feature_names[:1000])
tf_obj_word_1_1.tfidf_top_k(query, 20)

['آب' 'آباد' 'آبادان' 'آبادانی' 'آباده' 'آبادی' 'آبان' 'آبجو' 'آبخاز'
 'آبخوان' 'آبراه' 'آبراهام' 'آبراهه' 'آبشار' 'آبشارهای' 'آبله' 'آبکش'
 'آبگیری' 'آبی' 'آبیاری' 'آتش' 'آتشکده' 'آتلانتیک' 'آتن' 'آتواتوکا' 'آتی'
 'آث' 'آثار' 'آثارش' 'آثاری' 'آجر' 'آخر' 'آخری' 'آخرین' 'آداب' 'آدام'
 'آدامز' 'آدرس' 'آدلاید' 'آدم' 'آدمی' 'آدنوزین' 'آدهمار' 'آدولف' 'آدونیا'
 'آذر' 'آذربایجان' 'آذربایجانی' 'آذرماه' 'آذری' 'آذوقه' 'آر' 'آرا'
 'آراسته' 'آرام' 'آرامش' 'آرامگاه' 'آرامی' 'آرانی' 'آرای' 'آرایش' 'آرایه'
 'آرایی' 'آرتمیسینین' 'آرتور' 'آرد' 'آرزو' 'آرزوهای' 'آرزوی' 'آرسنال'
 'آرش' 'آرشیو' 'آرم' 'آرمان' 'آرواره' 'آرژانتین' 'آرژانتینی' 'آرکانزاس'
 'آریامهر' 'آریانا' 'آریایی' 'آزاد' 'آزادانه' 'آزادخان' 'آزادسازی'
 'آزادگان' 'آزادی' 'آزار' 'آزمایش' 'آزمایشگاه' 'آزمایشگاهی' 'آزمایشی'
 'آزمایی' 'آزمون' 'آسا' 'آسان' 'آسانسورهای' 'آسانی' 'آستارا' 'آستانه'
 'آسفالت' 'آسمان' 'آسمانی' 'آسوده' 'آسیا' 'آسیاب' 'آسیای' 'آسیایی' 'آسیب'
 'آسیبی' 'آش' 'آشامیدنی' 'آشتی' 'آشتیانی' 'آشفته' 'آشفتگی' 'آشنا' 'آشنایی'
 'آ

[484,
 482,
 349,
 343,
 345,
 344,
 466,
 481,
 347,
 341,
 463,
 1544,
 342,
 469,
 443,
 354,
 483,
 220,
 442,
 449]

###  Word Level: ngram (1,2)

In [19]:
tf_obj_word_1_2 = tfidf(docs=docs, ngram_range=(1,2), bpe_model=None)
tf_obj_word_1_2.tfidf_top_k(query, 20)

[484,
 482,
 345,
 343,
 344,
 349,
 481,
 466,
 341,
 347,
 342,
 354,
 1544,
 463,
 220,
 443,
 469,
 483,
 449,
 442]

### BPE: ngram (1,1)

In [20]:
tf_obj_bpe_1_1 = tfidf(docs=docs, ngram_range=(1,1), bpe_model=bpe)
# show 1000 features among 10000 selected features by tfidf 
print(tf_obj_bpe_1_1._feature_names[:1000])
tf_obj_bpe_1_1.tfidf_top_k(query, 20)

we are using bpe
['آ' 'آب' 'آباد' 'آمد' 'آن' 'آور' 'آوری' 'اب' 'ابان' 'ابت' 'ابتی' 'ابر'
 'ابری' 'ابع' 'ابق' 'ابل' 'ابله' 'ابه' 'ابی' 'اتر' 'اتری' 'اترین' 'اتل'
 'اتور' 'اتی' 'اتیک' 'اج' 'اجه' 'اح' 'احب' 'احت' 'احی' 'اخ' 'اخت' 'اخته'
 'اخر' 'اد' 'ادا' 'ادار' 'ادت' 'ادث' 'ادر' 'ادش' 'ادل' 'ادمی' 'اده' 'ادها'
 'ادهای' 'ادو' 'ادگی' 'ادی' 'ادیان' 'ادیه' 'ار' 'ارا' 'ارات' 'اران' 'ارت'
 'ارتی' 'ارد' 'اردی' 'ارستان' 'ارف' 'ارن' 'اره' 'ارها' 'ارهایی' 'ارک'
 'اری' 'اریان' 'از' 'ازد' 'ازده' 'ازدهم' 'ازم' 'ازند' 'ازه' 'ازی' 'اس'
 'اسا' 'اسب' 'است' 'استه' 'اسر' 'اسط' 'اسطه' 'اسم' 'اسپ' 'اسی' 'اسیون'
 'اش' 'اشا' 'اشت' 'اشته' 'اشد' 'اشی' 'اص' 'اصر' 'اصره' 'اصل' 'اصله' 'اض'
 'اضی' 'اط' 'اطر' 'اطع' 'اظ' 'اع' 'اعت' 'اعده' 'اعی' 'اعیل' 'اف' 'افت'
 'افظ' 'افع' 'افق' 'افه' 'اق' 'اقد' 'اقض' 'اقی' 'ال' 'الات' 'الب' 'الت'
 'التی' 'الح' 'الد' 'الدین' 'الع' 'العمل' 'الله' 'الم' 'اله' 'الی' 'الیا'
 'الیست' 'الیستی' 'الیسم' 'ام' 'اما' 'امبر' 'امت' 'امح' 'امحمد' 'امد'
 'امع' 'امل' 'امن' 'امه' 'امون' 'امی' 'امین' 

[484,
 482,
 349,
 344,
 347,
 345,
 343,
 466,
 481,
 341,
 342,
 442,
 463,
 1544,
 443,
 220,
 483,
 469,
 2080,
 354]

### BPE: ngram (1,2)

In [21]:
tf_obj_bpe_1_2 = tfidf(docs=docs, ngram_range=(1,2), bpe_model=bpe)
tf_obj_bpe_1_2.tfidf_top_k(query, 20)

we are using bpe


[484,
 482,
 349,
 344,
 442,
 347,
 343,
 342,
 345,
 466,
 341,
 1544,
 481,
 463,
 443,
 354,
 220,
 483,
 469,
 438]

### TFIDF Weighted

In [22]:
tf_obj_weighted = tfidf(docs=docs, ngram_range=(1,2), bpe_model=None)
tf_obj_weighted.tfidf_weighted_top_k(query, 20)

[341,
 466,
 443,
 484,
 481,
 1544,
 345,
 343,
 349,
 482,
 442,
 347,
 438,
 342,
 344,
 351,
 348,
 2727,
 354,
 465]

## Evaluation

In [66]:
# Create all models information as a dcit
models = {'Word Level: ngram (1,1)': {'model': tf_obj_word_1_1, 'MRR': 0, 'P@K': 0, 'MAP': 0},
          'Word Level: ngram (1,2)': {'model': tf_obj_word_1_2, 'MRR': 0, 'P@K': 0, 'MAP': 0},
          'BPE: ngram (1,1)': {'model': tf_obj_bpe_1_1, 'MRR': 0, 'P@K': 0, 'MAP': 0},
          'BPE: ngram (1,2)': {'model': tf_obj_bpe_1_2, 'MRR': 0, 'P@K': 0, 'MAP': 0},
          'TFIDF Weighted': {'model': tf_obj_weighted, 'MRR': 0, 'P@K': 0, 'MAP': 0}
         }

In [67]:
query_data = {}
with open(YAML_PATH, "r") as stream:
    try:
        query_data = dict(yaml.safe_load(stream))
    except yaml.YAMLError as exc:
        print(exc)

queries = list(query_data.keys())

In [68]:
TOP_K_FROM_MODELS = 100

In [69]:
# compute MMR metric for all models and queries
def MMR_TFIDf(models, queries, query_data):
    relevent_match_list = []
    for model in models.keys():
        relevent_match_list = []
        for query_i in tqdm(queries):
            if model == 'TFIDF Weighted':
                top_results_indices_i = models[model]['model'].tfidf_weighted_top_k(query_i, TOP_K_FROM_MODELS)
            else:
                top_results_indices_i = models[model]['model'].tfidf_top_k(query_i, TOP_K_FROM_MODELS)
            boolian_match = (np.array(top_results_indices_i) == query_data[query_i]['relevant'][0]).astype(int)
            relevent_match_list.append(list(boolian_match))
        models[model]['MRR'] = mean_reciprocal_rank(relevent_match_list)
    return models



In [70]:
# compute P@K metric for all models and queries
def PATK_TFIDF(models, queries, query_data, k=10):
    patk = []

    for model in models.keys():
        patk = []
        for query_i in tqdm(queries):
            if model == 'TFIDF Weighted':
                top_results_indices_i = models[model]['model'].tfidf_weighted_top_k(query_i, TOP_K_FROM_MODELS)
            else:
                top_results_indices_i = models[model]['model'].tfidf_top_k(query_i, TOP_K_FROM_MODELS)
            boolian_match = np.isin(top_results_indices_i, (query_data[query_i]['relevant'] + query_data[query_i]['similar_high'])).astype(int)
            patk.append(precision_at_k(boolian_match, k))
        models[model]['P@K'] = sum(patk)/len(patk)
    return models



In [71]:
# compute MAP metric for all models and queries
def MAP_TFIDF(models, queries, query_data):
    map_i = []

    for model in models.keys():
        map_i = []
        for query_i in tqdm(queries):
            if model == 'TFIDF Weighted':
                top_results_indices_i = models[model]['model'].tfidf_weighted_top_k(query_i, TOP_K_FROM_MODELS)
            else:
                top_results_indices_i = models[model]['model'].tfidf_top_k(query_i, TOP_K_FROM_MODELS)
            boolian_match = np.isin(top_results_indices_i, (query_data[query_i]['relevant'] + query_data[query_i]['similar_high'])).astype(int)
            map_i.append(list(boolian_match))

        models[model]['MAP'] = mean_average_precision(map_i)
    return models



In [72]:
models = MMR_TFIDf(models, queries, query_data)

100%|██████████| 150/150 [02:05<00:00,  1.20it/s]


In [73]:
models = PATK_TFIDF(models, queries, query_data, k=10)

100%|██████████| 150/150 [02:04<00:00,  1.21it/s]


In [74]:
models = MAP_TFIDF(models, queries, query_data)

100%|██████████| 150/150 [02:06<00:00,  1.19it/s]


In [75]:
models

{'BPE: ngram (1,1)': {'MAP': 0.7490796895906846,
  'MRR': 0.526570578803912,
  'P@K': 0.4646666666666665,
  'model': <__main__.tfidf at 0x7f7cb9423b50>},
 'BPE: ngram (1,2)': {'MAP': 0.7441753114500218,
  'MRR': 0.5326684454291043,
  'P@K': 0.46066666666666645,
  'model': <__main__.tfidf at 0x7f7cb7bd2a50>},
 'TFIDF Weighted': {'MAP': 0.5417913041777244,
  'MRR': 0.3925153095796033,
  'P@K': 0.34266666666666673,
  'model': <__main__.tfidf at 0x7f7cb7be9d90>},
 'Word Level: ngram (1,1)': {'MAP': 0.7574256642683204,
  'MRR': 0.5438906926406927,
  'P@K': 0.47333333333333333,
  'model': <__main__.tfidf at 0x7f7cb9423e10>},
 'Word Level: ngram (1,2)': {'MAP': 0.7460672160726832,
  'MRR': 0.5213573167690817,
  'P@K': 0.4673333333333333,
  'model': <__main__.tfidf at 0x7f7cb9446850>}}